In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [43]:
# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


#### 1. Data Loading

In [44]:
customers = pd.read_csv('../datasets/olist_customers_dataset.csv')
orders = pd.read_csv('../datasets/olist_orders_dataset.csv')
order_items = pd.read_csv('../datasets/olist_order_items_dataset.csv')
products = pd.read_csv('../datasets/olist_products_dataset.csv')
sellers = pd.read_csv('../datasets/olist_sellers_dataset.csv')
payments = pd.read_csv('../datasets/olist_order_payments_dataset.csv')
reviews = pd.read_csv('../datasets/olist_order_reviews_dataset.csv')
category_translation = pd.read_csv('../datasets/product_category_name_translation.csv')
geolocation = pd.read_csv('../datasets/olist_geolocation_dataset.csv')

#### 2. Basic Information

In [45]:
datasets = {
    'Customers': customers,
    'Orders': orders,
    'Order Items': order_items,
    'Products': products,
    'Sellers': sellers,
    'Payments': payments,
    'Reviews': reviews,
    'Category Translation': category_translation,
    'Geolocation': geolocation
}

print("\nDataset Shapes:")
for name, df in datasets.items():
    print(f"{name:25s}: {df.shape[0]:,} rows × {df.shape[1]} columns")



Dataset Shapes:
Customers                : 99,441 rows × 5 columns
Orders                   : 99,441 rows × 8 columns
Order Items              : 112,650 rows × 7 columns
Products                 : 32,951 rows × 9 columns
Sellers                  : 3,095 rows × 4 columns
Payments                 : 103,886 rows × 5 columns
Reviews                  : 99,224 rows × 7 columns
Category Translation     : 71 rows × 2 columns
Geolocation              : 1,000,163 rows × 5 columns


#### 3. Detailed data inspection

In [46]:
def inspect_dataframe(df, name):
    print(f"\n{'=' * 60}")
    print(f"{name.upper()}")
    print(f"{'=' * 60}")
    print(f"\nShape: {df.shape}")
    print(f"\nColumns and Data Types:")
    print(df.dtypes)
    print(f"\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0])
    print(f"\nFirst few rows:")
    print(df.head(3))
    print(f"\nBasic Statistics:")
    print(df.describe())

# Inspect each dataset
for name, df in datasets.items():
    inspect_dataframe(df, name)


CUSTOMERS

Shape: (99441, 5)

Columns and Data Types:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

Missing Values:
Empty DataFrame
Columns: [Missing Count, Percentage]
Index: []

First few rows:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  

Basic Statistics:
       customer_zip_code_prefix
count              99441.000000
mean       

#### 4. Data quality checks

In [47]:
# Check for duplicates
print("\nDuplicate Records:")
for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name:25s}: {duplicates:,} duplicates")

# Check unique values in key columns
print("\n\nUnique Values in Key Columns:")
print(f"Unique Customers: {customers['customer_unique_id'].nunique():,}")
print(f"Unique Orders: {orders['order_id'].nunique():,}")
print(f"Unique Products: {products['product_id'].nunique():,}")
print(f"Unique Sellers: {sellers['seller_id'].nunique():,}")
print(f"Unique Zip Codes: {geolocation['geolocation_zip_code_prefix'].nunique():,}")

# Order Status Distribution
print("\n\nOrder Status Distribution:")
print(orders['order_status'].value_counts())

# Payment Types Distribution
print("\n\nPayment Types Distribution:")
print(payments['payment_type'].value_counts())



Duplicate Records:
Customers                : 0 duplicates
Orders                   : 0 duplicates
Order Items              : 0 duplicates
Products                 : 0 duplicates
Sellers                  : 0 duplicates
Payments                 : 0 duplicates
Reviews                  : 0 duplicates
Category Translation     : 0 duplicates
Geolocation              : 261,831 duplicates


Unique Values in Key Columns:
Unique Customers: 96,096
Unique Orders: 99,441
Unique Products: 32,951
Unique Sellers: 3,095
Unique Zip Codes: 19,015


Order Status Distribution:
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64


Payment Types Distribution:
payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64


#### 5. Data Cleaning

In [48]:
# ORDERS - Keep missing timestamps

print("Missing timestamps by order status:")
missing_by_status = orders.groupby('order_status').agg({
    'order_approved_at': lambda x: x.isna().sum(),
    'order_delivered_carrier_date': lambda x: x.isna().sum(),
    'order_delivered_customer_date': lambda x: x.isna().sum()
})
print(missing_by_status)



Missing timestamps by order status:
              order_approved_at  order_delivered_carrier_date  \
order_status                                                    
approved                      0                             2   
canceled                    141                           550   
created                       5                             5   
delivered                    14                             2   
invoiced                      0                           314   
processing                    0                           301   
shipped                       0                             0   
unavailable                   0                           609   

              order_delivered_customer_date  
order_status                                 
approved                                  2  
canceled                                619  
created                                   5  
delivered                                 8  
invoiced                             

In [49]:
# PRODUCTS - Handle missing values
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [50]:
print("\n\nProducts: Handling missing values...")
print(f"Before: {products.isnull().sum().sum()} total missing values")

products['product_category_name'].fillna('unknown', inplace=True)

# Add 'unknown' to category translations if not present
if 'unknown' not in category_translation['product_category_name'].values:
    unknown_row = pd.DataFrame({
        'product_category_name': ['unknown'],
        'product_category_name_english': ['unknown']
    })
    category_translation = pd.concat([category_translation, unknown_row], ignore_index=True)
    print("Added 'unknown' category to translations")

products['product_name_lenght'].fillna(0, inplace=True)
products['product_description_lenght'].fillna(0, inplace=True)
products['product_photos_qty'].fillna(1, inplace=True)

products['product_weight_g'].fillna(products['product_weight_g'].median(), inplace=True)
products['product_length_cm'].fillna(products['product_length_cm'].median(), inplace=True)
products['product_height_cm'].fillna(products['product_height_cm'].median(), inplace=True)
products['product_width_cm'].fillna(products['product_width_cm'].median(), inplace=True)

print(f"After: {products.isnull().sum()} total missing values")



Products: Handling missing values...
Before: 2448 total missing values
Added 'unknown' category to translations
After: product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64 total missing values


In [51]:
# REVIEWS - Add flags for comments
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [52]:
print("\n\nReviews: Adding comment flags...")

# Add flags
reviews['has_title'] = reviews['review_comment_title'].notna()
reviews['has_message'] = reviews['review_comment_message'].notna()

# Fill nulls
reviews['review_comment_title'].fillna('', inplace=True)
reviews['review_comment_message'].fillna('', inplace=True)

print(f"Reviews with title: {reviews['has_title'].sum():,} ({reviews['has_title'].mean()*100:.1f}%)")
print(f"Reviews with message: {reviews['has_message'].sum():,} ({reviews['has_message'].mean()*100:.1f}%)")





Reviews: Adding comment flags...
Reviews with title: 11,568 (11.7%)
Reviews with message: 40,977 (41.3%)


In [53]:
# GEOLOCATION - Remove duplicates
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [54]:
# GEOLOCATION - Remove duplicates
print("\n\nGeolocation: Aggregating duplicates...")
print(f"Before: {len(geolocation):,} rows")

geolocation_clean = geolocation.groupby('geolocation_zip_code_prefix').agg({
    'geolocation_lat': 'mean',
    'geolocation_lng': 'mean',
    'geolocation_city': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'geolocation_state': 'first'
}).reset_index()

print(f"After: {len(geolocation_clean):,} unique zip codes")

# Replace original with cleaned version
geolocation = geolocation_clean



Geolocation: Aggregating duplicates...
Before: 1,000,163 rows
After: 19,015 unique zip codes


#### 6. Temporal Analysis

In [55]:
# Convert date columns to datetime
date_columns = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date',
                'order_estimated_delivery_date']

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Date range
print(f"\nOrder Date Range:")
print(f"First Order: {orders['order_purchase_timestamp'].min()}")
print(f"Last Order: {orders['order_purchase_timestamp'].max()}")
print(f"Time Span: {(orders['order_purchase_timestamp'].max() - orders['order_purchase_timestamp'].min()).days} days")

# Calculate delivery time
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - 
                                  orders['order_purchase_timestamp']).dt.days
# Delivery time statistics
print(f"Fastest Delivery: {orders['delivery_time_days'].min()} days")
print(f"Slowest Delivery: {orders['delivery_time_days'].max()} days")

print(f"\nDelivery Time Statistics:")
print(orders['delivery_time_days'].describe())


Order Date Range:
First Order: 2016-09-04 21:15:19
Last Order: 2018-10-17 17:30:18
Time Span: 772 days
Fastest Delivery: 0.0 days
Slowest Delivery: 209.0 days

Delivery Time Statistics:
count    96476.000000
mean        12.094086
std          9.551746
min          0.000000
25%          6.000000
50%         10.000000
75%         15.000000
max        209.000000
Name: delivery_time_days, dtype: float64


#### 7. Geographic analysis

In [56]:
# Top locations
print("\nTop 10 Customer States:")
print(customers['customer_state'].value_counts().head(10))

print("\nTop 10 Customer Cities:")
print(customers['customer_city'].value_counts().head(10))

print("\nTop 10 Seller States:")
print(sellers['seller_state'].value_counts().head(10))


Top 10 Customer States:
customer_state
SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
Name: count, dtype: int64

Top 10 Customer Cities:
customer_city
sao paulo                15540
rio de janeiro            6882
belo horizonte            2773
brasilia                  2131
curitiba                  1521
campinas                  1444
porto alegre              1379
salvador                  1245
guarulhos                 1189
sao bernardo do campo      938
Name: count, dtype: int64

Top 10 Seller States:
seller_state
SP    1849
PR     349
MG     244
SC     190
RJ     171
RS     129
GO      40
DF      30
ES      23
BA      19
Name: count, dtype: int64


#### 8. Product Analysis

In [57]:
# Merge products with category translation
products_translated = products.merge(
    category_translation, 
    on='product_category_name', 
    how='left'
)

print("\nTop 15 Product Categories:")
print(products_translated['product_category_name_english'].value_counts().head(15))

print("\nProduct Dimensions Statistics:")
print(products[['product_weight_g', 'product_length_cm', 
                'product_height_cm', 'product_width_cm']].describe())



Top 15 Product Categories:
product_category_name_english
bed_bath_table              3029
sports_leisure              2867
furniture_decor             2657
health_beauty               2444
housewares                  2335
auto                        1900
computers_accessories       1639
toys                        1411
watches_gifts               1329
telephony                   1134
baby                         919
perfumery                    868
fashion_bags_accessories     849
stationery                   849
cool_stuff                   789
Name: count, dtype: int64

Product Dimensions Statistics:
       product_weight_g  product_length_cm  product_height_cm  \
count      32951.000000       32951.000000       32951.000000   
mean        2276.376802          30.814725          16.937422   
std         4281.926387          16.914005          13.637175   
min            0.000000           7.000000           2.000000   
25%          300.000000          18.000000           8.000000   

#### 9. Payment Analysis

In [58]:
# Merge order items with products and payments
order_analysis = order_items.merge(orders, on='order_id')
order_analysis = order_analysis.merge(payments, on='order_id')

print("\nPrice Statistics:")
print(order_items['price'].describe())

print("\nFreight Statistics:")
print(order_items['freight_value'].describe())

print("\nPayment Value Statistics:")
print(payments['payment_value'].describe())

print("\nPayment Installments Distribution:")
print(payments['payment_installments'].value_counts().head(10))

# Calculate total order values
total_order_value = order_items.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()
total_order_value['total_value'] = (total_order_value['price'] + 
                                     total_order_value['freight_value'])

print("\nTotal Order Value Statistics:")
print(total_order_value['total_value'].describe())


Price Statistics:
count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
25%          39.900000
50%          74.990000
75%         134.900000
max        6735.000000
Name: price, dtype: float64

Freight Statistics:
count    112650.000000
mean         19.990320
std          15.806405
min           0.000000
25%          13.080000
50%          16.260000
75%          21.150000
max         409.680000
Name: freight_value, dtype: float64

Payment Value Statistics:
count    103886.000000
mean        154.100380
std         217.494064
min           0.000000
25%          56.790000
50%         100.000000
75%         171.837500
max       13664.080000
Name: payment_value, dtype: float64

Payment Installments Distribution:
payment_installments
1     52546
2     12413
3     10461
4      7098
10     5328
5      5239
8      4268
6      3920
7      1626
9       644
Name: count, dtype: int64

Total Order Value Statistics:
count    98666.000000
mean       160.577638
std

#### 10. Review Analysis

In [59]:
print("\nReview Score Distribution:")
print(reviews['review_score'].value_counts().sort_index())

print(f"\nAverage Review Score: {reviews['review_score'].mean():.2f}")
print(f"Median Review Score: {reviews['review_score'].median():.0f}")



Review Score Distribution:
review_score
1    11424
2     3151
3     8179
4    19142
5    57328
Name: count, dtype: int64

Average Review Score: 4.09
Median Review Score: 5


#### 11. Data Relationships

In [60]:
# Check relationship integrity
print("\nRelationship Checks:")

# Orders to Customers
orders_with_customers = orders.merge(customers, on='customer_id', how='left', indicator=True)
print(f"Orders with valid customers: {(orders_with_customers['_merge'] == 'both').sum():,} / {len(orders):,}")

# Order Items to Orders
items_with_orders = order_items.merge(orders, on='order_id', how='left', indicator=True)
print(f"Order items with valid orders: {(items_with_orders['_merge'] == 'both').sum():,} / {len(order_items):,}")

# Order Items to Products
items_with_products = order_items.merge(products, on='product_id', how='left', indicator=True)
print(f"Order items with valid products: {(items_with_products['_merge'] == 'both').sum():,} / {len(order_items):,}")

# Items per order
items_per_order = order_items.groupby('order_id').size()
print(f"\nItems per Order Statistics:")
print(items_per_order.describe())


Relationship Checks:
Orders with valid customers: 99,441 / 99,441
Order items with valid orders: 112,650 / 112,650
Order items with valid products: 112,650 / 112,650

Items per Order Statistics:
count    98666.000000
mean         1.141731
std          0.538452
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         21.000000
dtype: float64


#### 12. Save cleaned datasets (only modified ones)

In [61]:
# 1. Products - filled missing values
products.to_csv(f'../datasets/products_clean.csv', index=False)

# 2. Reviews - added flags and filled empty strings
reviews.to_csv(f'../datasets/reviews_clean.csv', index=False)

# 3. Geolocation - aggregated from 1M to 19K rows
geolocation.to_csv(f'../datasets/geolocation_clean.csv', index=False)

# 4. Orders - added delivery_time_days column + datetime conversion
orders.to_csv(f'../datasets/orders_clean.csv', index=False)

# 5. Category Translation - ensured 'unknown' category exists
category_translation.to_csv(f'../datasets/category_translation_clean.csv', index=False)
